In [1]:
import sys
sys.path.append(r"D:\往期\奇货居\ZERO")


import pymysql
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine


from tools import *

conn_d = {
    'user':"root",
    'password':'jiuguai',
    'host':'localhost',
    'port':3306,
    'database':'qhj',
    "charset" :"utf8mb4"
}

con = pymysql.connect(**conn_d)
cursor = con.cursor()



sql = 'select 订单号,收件人地址 from order_details where 国家 is null'
engine = create_engine('mysql+pymysql://{user}:{password}@{host}:{port}/{database}'.format(**conn_d),encoding='utf-8')
data = pd.read_sql(sql,engine)

data = completion_col(data,{'国家','省份','城市','县区','街道','lng','lat'})

In [2]:
con = pymysql.connect(**conn_d)
cursor = con.cursor()

In [3]:
class GetAddr():
    keys  = {
            "baidu":'pKthM9TxiqPsGx81IAwlkVuGRphmwdDR',
             "gaode": "138942d50e3bee60df440472c9ae123f"   
        }
         
    def __init__(self, api='baidu'):
        
        self.apis = {
            "baidu":[self.__baidu_url,self.__baidu_geo,self.__baidu_regeo],
            'gaode':[self.__gaode_url,self.__gaode_geo,self.__gaode_regeo]
        }
        
        self.set_ak(api=api)
        

    def set_ak(self,api='baidu'):
        self.api = api
        self.ak = self.__class__.keys[api]
        api_kit = self.apis.get(api,self.__baidu_url)
        api_kit[0]()
        self.__geo = api_kit[1]
        self.__regeo = api_kit[2]
    
    def __gaode_url(self):
        self.geo_url ="https://restapi.amap.com/v3/geocode/geo?address=%(address)s&key=" + self.ak
        self.regeo_url = "https://restapi.amap.com/v3/geocode/regeo?location={lng},{lat}&radius=1000&extensions=all&key=" + self.ak

    def __baidu_url(self):
        self.geo_url ="http://api.map.baidu.com/geocoding/v3/?address=%(address)s&ret_coordtype=gcj02ll&output=json&ak="+ self.ak +"&output=json"
        self.regeo_url =  "http://api.map.baidu.com/reverse_geocoding/v3/?ak="+ self.ak+"&output=json&coordtype=gcj02ll&location={lat},{lng}"
    
    
    
    def __gaode_geo(self):
        rep = requests.get(self.geo_url %{"address":self.addr})

        result = rep.json()

        if result['status'] == "0":
            raise ServiceError("%s ： \n无法获取坐标" %self.addr)
        self.location = result['geocodes'][0]['location'].split(',')
        self.lng = self.location[0]
        self.lat = self.location[1]
        
    
    
    def __baidu_geo(self):

        rep = requests.get(self.geo_url %{"address":self.addr})
        
        result = rep.json()
        if result['status']:
            raise ServiceError("%s ： \n无法获取坐标" %self.addr)
        self.location = result['result']['location']
        self.lat = self.location['lat']
        self.lng = self.location['lng']
    
    def geo(self, *args, **kargs):

        if len(args) == 1:
            self.addr = args[0]
        if 'address' in kargs:
            self.addr = kargs.get('address')
        
        if 'addr' not in self.__dict__:
            raise TypeError("miss address")
        
        self.__geo()
        return self


    def __gaode_regeo(self):
        req_params = {
    
        "lng":self.lng,
        "lat":self.lat,
        }

        url = self.regeo_url.format(**req_params)

        rep = requests.get(url)

        result = rep.json()
        
        if result['status'] == "0":
            raise ServiceError(",lng%s ,lat:%s , 服务器异常无法查询地址" %(self.lng,self.lat))


        address_component = result['regeocode']['addressComponent']
        street_number = address_component['streetNumber']
        street = street_number['street']
        formatted_address = result['regeocode']['formatted_address']
#         print(result)
        self.item = { '国家':address_component['country'], 
                '省份':address_component['province'],
                '城市':address_component['city'],  
                '县区':address_component['district'], 
                '街道':street_number['street'],
                'lng':self.lng,
                'lat':self.lat,
               '格式化地址':formatted_address}
    

    def __baidu_regeo(self):
        req_params = {
    
        "lng":self.lng,
        "lat":self.lat,
        }

        
        rep = requests.get(self.regeo_url.format(**req_params))
        result = rep.json()
        
        if result['status']:
            raise ServiceError("lat:%s, lng%s 服务器异常无法查询地址" %(self.lat,self.lng))
            
        result = result['result']
        address_component = result['addressComponent']

        formatted_address = result['formatted_address']
#         print(result)
        self.item = { '国家':address_component['country'], 
                '省份':address_component['province'],
                '城市':address_component['city'],  
                '县区':address_component['district'], 
                '街道':address_component['street'],
                     'lng':self.lng,
                'lat':self.lat,
                
               '格式化地址':formatted_address}


    def regeo(self, *args, **kargs):
        
        if len(args) == 2:
            self.lng = args[0]
            self.lat = args[1]
        if 'lng' in kargs and 'lat' in kargs:
            self.lng = kargs.get('lng',self.lng)
            self.lat = kargs.get('lat',self.lat)
            
        if not ('lng' in self.__dict__ and 'lat'in self.__dict__):
            raise TypeError("miss lng or lat")
        
        self.__regeo()
        
        return self
    
    

gd = GetAddr('gaode')
gd.geo('长沙芙蓉区万象新天8楼815').regeo().item




{'国家': '中国',
 '省份': '湖南省',
 '城市': '长沙市',
 '县区': '芙蓉区',
 '街道': '车站北路',
 'lng': '113.011438',
 'lat': '28.200725',
 '格式化地址': '湖南省长沙市芙蓉区湘湖街道万象企业公馆5栋万象新天'}

In [4]:
for index,row in data.iterrows():
    address= row['收件人地址']

    addrs_component = gd.geo(address).regeo().item
    data.loc[index, '国家'] = addrs_component['国家']
    data.loc[index, '省份'] = addrs_component['省份']
    data.loc[index, '城市'] = addrs_component['城市']
    data.loc[index, '县区'] = addrs_component['县区']
    data.loc[index, '街道'] = addrs_component['街道']

    data.loc[index, 'lat'] = addrs_component['lat']
    data.loc[index, 'lng'] = addrs_component['lng']

In [5]:

def d(x,*args,**kargs):
    if x.name in args:
        x = x.apply(lambda y:None if isinstance(y,list) else y)
    return x
data = data.apply(d,args=('城市','县区','街道'))

In [6]:
data.to_sql('update_address',engine,index=False,if_exists='replace')

In [7]:
try:
    cursor.callproc('proc_update_address')
    con.commit()
except:
    pass
finally:
    con.close()

In [8]:
data

,订单号,收件人地址,lng,街道,城市,县区,国家,lat,省份
0,20191023184417285442,长沙市高新区金桥路10号长沙鑫航机轮刹车有限公司,112.843914,G5513长张高速,长沙市,岳麓区,中国,28.240931,湖南省
1,20191023184715474057,益阳市赫山区康富北路电大巷8号益阳广播电视大学,112.352063,康富北路,益阳市,赫山区,中国,28.567779,湖南省
2,20191023185702175707,湖南省长沙市宁乡县灰汤镇泉江村上杨柳组,112.335584,迎宾路,长沙市,宁乡市,中国,27.987372,湖南省
3,20191023192842376908,北京市昌平区北七家镇温泉花园B区70-361,116.417950,立汤路,None,昌平区,中国,40.126365,北京市
4,20191023192892376908,北京市昌平区北七家镇温泉花园B区70-361,116.417950,立汤路,None,昌平区,中国,40.126365,北京市
5,20191024001845224303,广东省广州市天河区员村北社大塘南15号,113.358121,员村二横路,广州市,天河区,中国,23.114715,广东省
6,20191024001895224303,广东省广州市天河区员村北社大塘南15号,113.358121,员村二横路,广州市,天河区,中国,23.114715,广东省
7,20191024004953517373,浙江省杭州市滨江区网商路699号阿里巴巴,120.190528,网商路,杭州市,滨江区,中国,30.189578,浙江省
8,20191024025952454982,地址湖南省长沙市宁乡县双凫铺镇汽车站周志佳兽药店,112.553625,民兴路,长沙市,宁乡市,中国,28.249986,湖南省
9,20191024033453222219,湖南省长沙市宁乡市喻家坳乡涌泉山村泥秋组7号,112.290475,None,长沙市,宁乡市,中国,28.188414,湖南省
